<img src="https://miro.medium.com/max/625/1*ala8WX2z47WYpn932hUkhA.jpeg"/>

The `SVM (Supported Vector Machine)` is a <b>supervised machine learning</b> algorithm typically used for <b>`binary classification`</b> . It's trained by feeding a dataset with <i>labeled examples (x, y)</i>. For instance, if your examples are email messages and it is a spam detection, then:
- An example message `xi` is defined as an <i>n-dimensional feature vector</i> that can be plotted on n-dimensional space.
- The feature vector, as name examples contains features (like count, like count etc.) of your email message in numerical form.
- Each feature vector is labeled with a class `yi`
- The class `yi` can either be a +ve and -ve

Using this algorithm `SVM` finds a `hyperplane` <b>(Decision Boundary)</b> which should ideally have the following properties
- It creates seperation between examples of two classes with a maximum margin.
- Its equation `(w.x + b = 0)` yields the value >= 1 for examples from +ve class and <=-1 for examples from -ve class.


### How does it find this hyperplane? 

By finding the optimal values `w* (weights/normal) and b* (intercept)` which define the hyperplane. The optimal values are found by `minimizing a cost function`. Once the Algorithm identifies these optimal values, the `SVM model` is then defined as :

<img src="https://miro.medium.com/max/265/1*pzms4wJY9_TeMFvAiTvjVA.png"/>

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm # For finding P-value
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

### Reading Dataset

In [2]:
data = pd.read_csv('BreastCancerDataset.csv')

# As SVM can only accept numerical values, so we transform the cateogories M and B 
# into values 1 and -1 respectively.
diagnosis_map = {'M': 1, 'B': -1}
data['diagnosis'] = data['diagnosis'].map(diagnosis_map)

### Feature Engineering

In [3]:
# Features and output columns in different DataFrames
Y = data.loc[:, 'diagnosis']
X = data.iloc[:, 1:] 

# Normalize the Features using MinMaxScaler
X_normalized = MinMaxScaler().fit_transform(X.values)
X = pd.DataFrame(X_normalized)

C:\ProgramData\Anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:373: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\ProgramData\Anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:374: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [4]:
# Split Dataset

# Insert 1 in every row for intercept b -> This is because of the cost Function
X.insert(loc=len(X.columns), column='intercept', value=1)

# Split
print('Splitting dataset..')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Splitting dataset..


### Cost Function
<img src="https://miro.medium.com/max/500/1*vn2HDrdqBsKN5rYw7rjO5w.png"/>

In SVM, our objective is to find `hyperplane` that seperates `+ve` and `-ve` examples with the largest margin while keeping the misclassification as low as possible.

So, we achieve this objective by minimizing the cost function:
<img src="https://miro.medium.com/max/688/1*JAS6rUTO7TDlrv4XZSMbsA.png"/>
`In the training phase, Larger C results in the narrow margin (for infinitely large C the SVM becomes hard margin) and smaller C results in the wider margin.`  
              
              
<b>Another version</b> of this cost function is: 
<img src="https://miro.medium.com/max/678/1*6w_B_DjhGvaqCnvhzhhkDg.png"/>
`Larger λ gives a wider margin and smaller λ results in the narrow margin (for infinitely small λ the SVM becomes hard margin).`

Here `λ` is `1/C` and so has the opposite effect. We can use any of them keeping in mind what each `regularization parameter (C and λ)` does.

In [5]:
def compute_cost(W, X, Y):
    # Calculate Hinge Loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0 # Equivalent to max(0, distance)
    hinge_loss = reg_strength * (np.sum(distances) / N)
    
    # Calculate Loss
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

Here the intercept term `b` is missing, as we pushed it to the weight vector 
<img src="https://miro.medium.com/max/868/1*jZJuvalHKsr18E5NUsemvQ.png"/>
For this reason, we added an extra column with all `1s` before splitting dataset.

### The Gradient of  the Cost Function
<img src="https://miro.medium.com/max/866/1*ww3F21VMVGp2NKhm0VTesA.png"/>

In [6]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    # If only one example is passed (eg. In case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])
        
    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))
    
    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (reg_strength * Y_batch[ind] * X_batch[ind])
        dw += di
        
    dw = dw / len(Y_batch) # Average
    return dw

### Training using SGD

We have to first minimize the cost function, look at the equation. To find `J(W)` minimum, we have to:
- Minimize `||w||**2` which maximizes margin `(2 / ||w||)`
- Minimize the sum of `hinge loss` which minimizes misclassifications.

The Hinge Loss Function

<img src="https://miro.medium.com/max/823/1*oKqlv9dGoyEhmvM3zCbsJA.png"/>

So, we minimize the cost function using `Stochastic Gradient Descent`. 
<img src="https://miro.medium.com/max/875/1*ykRTMpIdFqmyvTY6aEVoVw.png"/>

It works:
- Find the gradient of cost function i.e. `∇J(w’)`
- Move opposite to the gradient by a certain rate i.e. `w’ = w’ — ∝(∇J(w’))`
- Repeat step 1–3 until convergence i.e we found `w’` where `J(w)` is smallest.

In [7]:
def sgd(features, outputs):
    max_epochs = 1000
    weights = np.zeros(features.shape[1])
    
    for epoch in range(1, max_epochs):
        # Shuffle
        X, Y = shuffle(features, outputs)
        for idx, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[idx])
            weights -= (lr * ascent)
        
    return weights

In [8]:
# Stoppage criterion for SGD
def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)
        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is:{} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights

In [9]:
def train():
    print('Training started....')
    W = sgd(X_train.to_numpy(), y_train.to_numpy())
    print('Training finished')
    print('Weights are: {}'.format(W))

In [10]:
def test():
    Y_pred = np.array([])
    for i in range(X_test.shape[0]):
        yp = np.sign(np.dot(W, X_test.to_numpy()[i]))
        Y_pred = np.append(Y_pred, yp)
        
    print("accuracy on test dataset: {}".format(accuracy_score(y_test.to_numpy(), y_test_predicted)))
    print("recall on test dataset: {}".format(recall_score(y_test.to_numpy(), y_test_predicted)))
    print("precision on test dataset: {}".format(recall_score(y_test.to_numpy(), y_test_predicted)))

In [ ]:
reg_strength = 10000
learning_rate = 0.0001
train()

In [15]:
%run SVM_From_Scratch.py

reading dataset...
applying feature engineering...
splitting dataset into train and test sets...
training started...
Epoch is: 1 and Cost is: 7302.322568152557
Epoch is: 2 and Cost is: 6571.31677923402
Epoch is: 4 and Cost is: 5474.704523447078
Epoch is: 8 and Cost is: 3873.4330086323957
Epoch is: 16 and Cost is: 2652.50814625737
Epoch is: 32 and Cost is: 1980.2709881452106
Epoch is: 64 and Cost is: 1592.5884211331665
Epoch is: 128 and Cost is: 1324.3142639575485
Epoch is: 256 and Cost is: 1159.0037758357596
Epoch is: 512 and Cost is: 1074.5082761866936
Epoch is: 1024 and Cost is: 1048.4046400310142
Epoch is: 2048 and Cost is: 1040.4527930848028
training finished.
weights are: [ 3.53935971 11.05690531 -2.29300674 -7.90673076 10.1510758  -1.28424732
 -6.4438179   2.24260591 -3.88982507  3.24470575  4.93070569  4.82524927
 -4.73341036]
testing the model...
accuracy on test dataset: 0.9912280701754386
recall on test dataset: 0.9767441860465116
precision on test dataset: 0.9767441860465116